In [8]:
work_ids = set(Work.objects.values_list('id', flat=True))

In [9]:
import os
poster_ids = set(map(lambda x: int(x[:-4]), os.listdir('../mangaki/mangaki/static/img/posters')))

len(work_ids - poster_ids)  # Nombre de works qui n'ont pas de poster en local

1707

In [10]:
len(poster_ids - work_ids)  # Nombre de posters en local qui ne sont associés à aucun work (à cause des fusions)

44

Le code suivant essaie de télécharger les posters des anime qui n'en ont pas en local. Si l'URL est 404, il essaie de rafraîchir le poster.

In [5]:
import os
from collections import Counter
from urllib.request import urlretrieve
from mangaki.utils.db import refresh_poster

POSTER_DIR = '../mangaki/mangaki/static/img/posters/'

for work in Work.objects.exclude(poster='').exclude(poster__startswith='/')[:5]:
    print(work.id, work.poster)
    if os.path.isfile('../mangaki/mangaki/static/img/posters/%d.jpg' % work.id):
        work.poster = '/static/img/posters/%d.jpg' % work.id
        work.save()
    else:
        poster_url = work.poster
        try:  # Try to download the poster
            urlretrieve(work.poster, os.path.join(POSTER_DIR, '%d.jpg' % work.id))
        except:
            print('=> 404', poster_url)
            refresh_poster(work)
            try:  # Try to refresh the poster from the title
                urlretrieve(work.poster, os.path.join(POSTER_DIR, '%d.jpg' % work.id))
            except:
                print('=> 404 derechef', work.poster)
                work.poster = ''
                work.save()
                continue
        work.poster = '/static/img/posters/%d.jpg' % work.id
        work.save()

10042 http://cdn.myanimelist.net/images/anime/7/72651.jpg
=> 404 http://cdn.myanimelist.net/images/anime/7/72651.jpg
http://cdn.myanimelist.net/images/anime/11/75376.jpg for My Teen Romantic Comedy SNAFU TOO!
10061 http://cdn.myanimelist.net/images/anime/9/6434.jpg
=> 404 http://cdn.myanimelist.net/images/anime/9/6434.jpg
http://cdn.myanimelist.net/images/anime/7/74024.jpg for To Heart 2 OVA
10500 http://cdn.myanimelist.net/images/anime/3/28669.jpg
=> 404 http://cdn.myanimelist.net/images/anime/3/28669.jpg
http://cdn.myanimelist.net/images/anime/9/73974.jpg for SKET Dance
10281 http://cdn.myanimelist.net/images/anime/11/40711.jpg
=> 404 http://cdn.myanimelist.net/images/anime/11/40711.jpg
http://cdn.myanimelist.net/images/anime/10/73952.jpg for Hayate the Combat Butler: Can't Take My Eyes Off You
10303 http://cdn.myanimelist.net/images/anime/13/47445.jpg
=> 404 http://cdn.myanimelist.net/images/anime/13/47445.jpg
http://cdn.myanimelist.net/images/anime/9/73733.jpg for Umbrella


In [11]:
Counter(Work.objects.values_list('category', flat=True))  # Nombre d'œuvres de chaque catégorie

Counter({1: 10318, 2: 3857, 3: 1})

In [14]:
Work.objects.filter(category=2, poster__startswith='/').count()  # Nombre de mangas ayant un poster en local

3833

## Modifications manuelles

In [19]:
Work.objects.filter(id=6997).update(poster='http://www.manga-news.com/public/images/series/quatre-filles-docteur-march-nobi.jpg')  # 4 filles du Dr March

1

In [22]:
Work.objects.filter(poster='http://www.manga-news.com/public/images/series/').update(poster='')

0

In [28]:
Work.objects.filter(poster='')[0].reference_set.all()[0].__dict__  # Exemple de récupération de référence

{'_state': <django.db.models.base.ModelState at 0x104749630>,
 '_work_cache': <Work: D.C.S.S. ~Da Capo Second Season~>,
 'id': 7504,
 'url': 'http://myanimelist.net/anime/291',
 'work_id': 153}

In [17]:
Work.objects.filter(poster='').count()  # Nombre de mangas sans information de poster

13